In [0]:

--1. Average Age by Attrition
SELECT  
      HR.ATTRITION,
      ROUND(AVG(HR.AGE),2) AVG_AGE,
      ROUND(STD(HR.AGE),2) STD_AGE
      FROM workspace.default.ibm_hr HR
      GROUP BY HR.ATTRITION ;     
/*
OUTPUT:
ATTRITION | AVG_AGE | STD_AGE
Yes	      | 33.61   |	9.69
No	      | 37.56	  | 8.89
 */
 
--2. Leaver Frquency of Education Fields 
SELECT
    HR.EducationField,
    SUM(CASE WHEN HR.Attrition = 'Yes' THEN 1 ELSE 0 END) AS LeaversInField,
    COUNT(HR.EducationField) AS TotalInField,
   ROUND( (CAST(SUM(CASE WHEN HR.Attrition = 'Yes' THEN 1 ELSE 0 END) AS DOUBLE) * 100) / COUNT(HR.EducationField),2) AS PercentageOfLeavers
FROM
    workspace.default.ibm_hr HR
GROUP BY
    HR.EducationField
ORDER BY
    LeaversInField DESC;
/*
OUTPUT:
EducationField   | LeaversInField |	TotalInField | PercentageOfLeavers
Life Sciences	   | 89	            | 606	         | 14.69
Medical	         | 63	            | 464	         | 13.58
Marketing	       | 35	            | 159	         | 22.01
Technical Degree |	32	          | 132	         | 24.24
Other            |	11	          |  82          | 13.41
Human Resources  |	7	            |  27	         | 25.93
 */


/* 
  - This query compares each employee's Monthly Income to the Average Income of their Department, while also calculating the employee's Income Rank within their JobRole.

  - The following SQL query uses two different Window Functions:

    - ROUND(AVG(MonthlyIncome) OVER (PARTITION BY Department),2) : An Aggregate Window Function to calculate the average income by department.

    - DENSE_RANK() OVER (PARTITION BY JobRole ORDER BY MonthlyIncome DESC) : A Ranking Window Function to rank income by position.
*/
WITH RankedEmployees AS (
    SELECT
        HR.EmployeeNumber,
        HR.Department,
        HR.JobRole,
        HR.MonthlyIncome,
        
        ROUND(AVG(MonthlyIncome) OVER (PARTITION BY Department), 2) AS DepartmentAverageIncome,
        DENSE_RANK() OVER (PARTITION BY JobRole ORDER BY MonthlyIncome DESC) AS JobRoleIncomeRank,
        ROW_NUMBER() OVER (PARTITION BY Department ORDER BY MonthlyIncome DESC) AS DeptJobRoleRank
    FROM
        workspace.default.ibm_hr HR
)

SELECT
    EmployeeNumber,
    Department,
    JobRole,
    MonthlyIncome,
    DepartmentAverageIncome,
    JobRoleIncomeRank,
    DeptJobRoleRank
FROM
    RankedEmployees
WHERE
    DeptJobRoleRank <= 3
ORDER BY
    Department,
    JobRole,
    DeptJobRoleRank;

/*
Output : 
EmployeeNumber |	Department                    |	JobRole	           | MonthlyIncome | DepartmentAverageIncome | JobRoleIncomeRank | DeptJobRoleRank
1338	         |   Human Resources	            |  Manager	         | 19717	       | 6654.51	               |    8	             | 1
1625	         |   Human Resources	            |  Manager	         | 19658	       | 6654.51	               |    9	             | 2
1973	         |   Human Resources	            |  Manager	         | 19636	       | 6654.51	               |   10	             | 3
259	           |   Research & Development	      |  Manager	         | 19999         |	6281.25	               |    1	             | 1
1191	         |   Research & Development	      |  Manager	         | 19943	       | 6281.25	               |    2	             | 3
1035	         |   Research & Development	      |  Research Director | 19973	       | 6281.25	               |   1	             | 2
1282	         |   Sales	                      |  Manager	         | 19847         | 	6959.17	               |   5	             | 1
1038	         |   Sales	                      |  Manager	         | 19845         | 	6959.17	               |   6	             | 2  
1740	         |   Sales	                      |  Manager	         | 19833	       | 6959.17	               |   7               | 3

*/
